In [38]:
import pandas as pd
import numpy as np
import sys
import os
import json
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

from utils import *

aspects = [ 'actionability', 'grounding_specificity', 'verifiability', 'helpfulness', "professional_tone", 'valid_point', 'addressed_to_author']
silver_data = pd.read_csv('/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/human_annotation_gathered/all_human_annotations_processed.csv')



In [39]:
final_data = {
}
## only consider data starting from batch 4, as this is where we have only 3 annotators
silver_data = silver_data[silver_data['batch'] >3 ]
## only consider the silver data
for aspect in aspects:
    not_valid_points = 0
    final_data[aspect] = {
          'boda' : [],
    '6686ebe474531e4a1975636f' : [],
    '6740484e188a64793529ee77' : [],
    }

    ## only consider the silver data
    # aspect_silver_data = silver_data
    aspect_silver_data = silver_data[silver_data[f'{aspect}_label_type'] == 'silver']
    for i, row in aspect_silver_data.iterrows():
        data = row[aspect].replace("'", "\"")
        data = json.loads(data)
        labels = data['labels']
        annotators = data['annotators']
        for label, annotator in zip(labels, annotators):
            found = False
            if len(annotators) != 3:
                not_valid_points += 1
                continue
            ## search for the annotator in the unique id map

            for key in final_data[aspect]:
                if annotator in annotators_unique_id_batch_id_map[key]:
                    final_data[aspect][key].append(label)
                    found = True
                    
            if not found:
                print(f'annotator {annotator} not found in the unique id map')
               
    print(f'not valid points for {aspect} is {not_valid_points}')
        


not valid points for actionability is 54
not valid points for grounding_specificity is 34
not valid points for verifiability is 28
not valid points for helpfulness is 32
not valid points for professional_tone is 84
not valid points for valid_point is 58
not valid points for addressed_to_author is 102


In [40]:
prinaple_name = {
    'boda' : 'boda',
    '6686ebe474531e4a1975636f' : 'Annotator_X',
    '6740484e188a64793529ee77' : 'Annotator_Y',
}
for aspect in aspects:
    kappa_values = []
    
    ## calculate the agreement between each pair of annotators
    annotators = ['boda', '6686ebe474531e4a1975636f', '6740484e188a64793529ee77']
    print(f'Aspect: {aspect}')
    print(f'Number of data points: {len(final_data[aspect][annotators[0]])}')
    for i in range(len(annotators)):
        for j in range(i+1, len(annotators)):
            annotator1 = annotators[i]
            annotator2 = annotators[j]
            data1 = final_data[aspect][annotator1]
            data2 = final_data[aspect][annotator2]

            annotator1 = prinaple_name[annotator1]
            annotator2 = prinaple_name[annotator2]
            if aspect in ['actionability', 'grounding_specificity', 'helpfulness']:
                _, _, _, _, kappa_quadratic, _ = get_stats(data1, data2, aspect)
                print(f'Aspect: {aspect}, Annotators: {annotator1} & {annotator2}, Quadratic Kappa: {round(kappa_quadratic, 2)}')
                kappa_values.append(kappa_quadratic)

            elif aspect == 'verifiability':
                _, _, _, _, kappa_quadratic, _, f1_X = get_stats(data1, data2, aspect)
                print(f'Aspect: {aspect}, Annotators: {annotator1} & {annotator2}, Quadratic Kappa: {round(kappa_quadratic, 2)}')
                print(f'Aspect: {aspect}, Annotators: {annotator1} & {annotator2}, F1_X: {round(f1_X, 2)}')
                kappa_values.append(kappa_quadratic)

            else:
                f1 = get_stats(data1, data2, aspect)
                print(f'Aspect: {aspect}, Annotators: {annotator1} & {annotator2}, F1: {round(f1, 2)}')




    if kappa_values:
        avg_kappa = sum(kappa_values) / len(kappa_values)
        print(f'Aspect: {aspect}, Average Quadratic Kappa: {round(avg_kappa, 2)}\n')


Aspect: actionability
Number of data points: 397
Aspect: actionability, Annotators: boda & Annotator_X, Quadratic Kappa: 0.67
Aspect: actionability, Annotators: boda & Annotator_Y, Quadratic Kappa: 0.5
Aspect: actionability, Annotators: Annotator_X & Annotator_Y, Quadratic Kappa: 0.39
Aspect: actionability, Average Quadratic Kappa: 0.52

Aspect: grounding_specificity
Number of data points: 367
Aspect: grounding_specificity, Annotators: boda & Annotator_X, Quadratic Kappa: 0.2
Aspect: grounding_specificity, Annotators: boda & Annotator_Y, Quadratic Kappa: 0.13
Aspect: grounding_specificity, Annotators: Annotator_X & Annotator_Y, Quadratic Kappa: 0.37
Aspect: grounding_specificity, Average Quadratic Kappa: 0.23

Aspect: verifiability
Number of data points: 403
Aspect: verifiability, Annotators: boda & Annotator_X, Quadratic Kappa: 0.49
Aspect: verifiability, Annotators: boda & Annotator_X, F1_X: 0.59
Aspect: verifiability, Annotators: boda & Annotator_Y, Quadratic Kappa: 0.62
Aspect: ver